In [2]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import datetime
# import os

# def process_file(file):
#     col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
#     for i in range(1, 33):
#         col_names.append(f"Sp{i}")
    
#     df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
#     df = df.set_index('Index')
#     df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
#     df = df[df["MonStatus"] == 1]

#     month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
#     df['DateM'] = df['DateM'].str[:3].map(month_map)
#     df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
#     df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

#     df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
#                                      month=df['Date'].dt.month,
#                                      day=df['Date'].dt.day,
#                                      hour=df['Time'].dt.hour,
#                                      minute=df['Time'].dt.minute,
#                                      second=df['Time'].dt.second))

#     df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

#     day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
#     df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
#     return df

# def process_files(files):
#     dataframes = {}
#     for idx, file in enumerate(files, start=1):
#         df = process_file(file)
#         dataframes[f'df{idx}'] = df
#     return dataframes

# def filter_and_merge(dataframes, threshold=0):
#     merged_filtered_dfs = []
#     for name, df in dataframes.items():
#         additional_columns = df.columns[:3]
#         filtered_dfs = []

#         for day in range(1, 9):  
#             day_df = df[df['Day'] == day] 
#             count_mov = day_df.filter(like="Sp")  
#             sum_mov = count_mov.sum(axis=0)  
#             filter_mov = sum_mov > threshold  
#             columns_to_keep = filter_mov[filter_mov].index 

#             all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
#             filtered_df = day_df[all_columns_to_keep]
#             filtered_dfs.append(filtered_df)

#         merged_df = pd.concat(filtered_dfs)
#         merged_df1 = merged_df.dropna(axis=1)
#         merged_filtered_dfs.append(merged_df1)
    
#     final_merged_df = pd.concat(merged_filtered_dfs)
#     return final_merged_df

# def entrainment(data, column):
#     if column not in data.columns:
#         return False
    
#     dflight = data[data['Light'] == 1][column]
#     dfdark = data[data['Light'] == 0][column]
    
#     lightmean = np.mean(dflight)
#     darkmean = np.mean(dfdark)
    
#     if darkmean == 0:
#         return False
    
#     diff = lightmean / darkmean
#     return diff > 0.25

# files = [
#     'Steatoda A masking 02 pm.txt', 'Steatoda A masking 10 am.txt',
#     'Steatoda A masking 4 am.txt', 'Steatoda A masking midnight.txt'
# ]

# dataframes = process_files(files)
# final_merged_df = filter_and_merge(dataframes, threshold=0)

# # Check for entrainment
# entrained_spiders = []
# for spider in final_merged_df.columns[3:]:  # Skip the first three columns
#     if entrainment(final_merged_df, spider):
#         entrained_spiders.append(spider)

# # Filter the final dataframe to keep only entrained spiders
# final_merged_df1 = final_merged_df[['Day', 'Time', 'Light'] + entrained_spiders]
# final_merged_df2 = final_merged_df1.dropna(axis='columns')

# cols_list = final_merged_df2.columns.tolist()
# final_list = cols_list[3:]

# # Print the result
# print("The spiders in group A that have data across all times and are entrained are:", final_list)

The spiders in group A that have data across all times and are entrained are: ['Sp4', 'Sp9', 'Sp10', 'Sp12', 'Sp14', 'Sp15', 'Sp19', 'Sp21']


In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

def process_file(file):
    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]

    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    return df

def process_files(files):
    dataframes = {}
    for idx, file in enumerate(files, start=1):
        df = process_file(file)
        dataframes[f'df{idx}'] = df
    return dataframes

def filter_and_merge(dataframes, threshold=0):
    merged_filtered_dfs = []
    for name, df in dataframes.items():
        additional_columns = df.columns[:3]
        filtered_dfs = []

        for day in range(1, 9):  
            day_df = df[df['Day'] == day] 
            count_mov = day_df.filter(like="Sp")  
            sum_mov = count_mov.sum(axis=0)  
            filter_mov = sum_mov > threshold  
            columns_to_keep = filter_mov[filter_mov].index 

            all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
            filtered_df = day_df[all_columns_to_keep]
            filtered_dfs.append(filtered_df)

        merged_df = pd.concat(filtered_dfs)
        merged_df1 = merged_df.dropna(axis=1)
        merged_filtered_dfs.append(merged_df1)
    
    final_merged_df = pd.concat(merged_filtered_dfs)
    return final_merged_df

def entrainment(data, column):
    dflight = data[data['Light'] == 1][column]
    dfdark = data[data['Light'] == 0][column]
    
    lightmean = np.mean(dflight)
    darkmean = np.mean(dfdark)
    diff = lightmean / darkmean
    
    return diff > 0.25

files = ['Steatoda B masking 04 pm.txt', 'Steatoda B masking 10 pm.txt', 'Steatoda B masking 12 pm.txt', 'Steatoda B maskng 2am.txt']

dataframes = process_files(files)
final_merged_df = filter_and_merge(dataframes, threshold=0)

entrained_spiders = []
for spider in final_merged_df.columns[3:]:
    if entrainment(final_merged_df, spider):
        entrained_spiders.append(spider)

final_merged_df1 = final_merged_df[['Day', 'Time', 'Light'] + entrained_spiders]
final_merged_df2 = final_merged_df1.dropna(axis='columns')

cols_list = final_merged_df2.columns.tolist()
final_list = cols_list[3:]

print("The spiders in group B that have data across all times and are entrained are:", final_list)

The spiders in group B that have data across all times and are entrained are: ['Sp1', 'Sp3', 'Sp5', 'Sp7', 'Sp12', 'Sp14', 'Sp17', 'Sp18', 'Sp19', 'Sp21', 'Sp22', 'Sp23']


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

def process_file(file):
    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]

    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    return df

def process_files(files):
    dataframes = {}
    for idx, file in enumerate(files, start=1):
        df = process_file(file)
        dataframes[f'df{idx}'] = df
    return dataframes

def filter_and_merge(dataframes, threshold=0):
    merged_filtered_dfs = []
    for name, df in dataframes.items():
        additional_columns = df.columns[:3]
        filtered_dfs = []

        for day in range(1, 9):  
            day_df = df[df['Day'] == day] 
            count_mov = day_df.filter(like="Sp")  
            sum_mov = count_mov.sum(axis=0)  
            filter_mov = sum_mov > threshold  
            columns_to_keep = filter_mov[filter_mov].index 

            all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
            filtered_df = day_df[all_columns_to_keep]
            filtered_dfs.append(filtered_df)

        merged_df = pd.concat(filtered_dfs)
        merged_df1 = merged_df.dropna(axis=1, how='all')  # Drop columns that are all NaN
        merged_filtered_dfs.append(merged_df1)
    
    final_merged_df = pd.concat(merged_filtered_dfs)
    return final_merged_df

def entrainment(data, column):
    if column not in data.columns:
        return False
    
    dflight = data[data['Light'] == 1][column]
    dfdark = data[data['Light'] == 0][column]
    
    lightmean = np.mean(dflight)
    darkmean = np.mean(dfdark)
    
    if darkmean == 0:
        return False
    
    diff = lightmean / darkmean
    return diff > 0.25

files = ['Steatoda B masking 04 pm.txt', 'Steatoda B masking 10 pm.txt', 'Steatoda B masking 12 pm.txt', 'Steatoda B maskng 2am.txt']

dataframes = process_files(files)
final_merged_df = filter_and_merge(dataframes, threshold=0)

entrained_spiders = []
for spider in final_merged_df.columns[3:]:
    if entrainment(final_merged_df, spider):
        entrained_spiders.append(spider)

# Create the final dataframe with only entrained spiders
final_merged_df1 = final_merged_df[['Day', 'Time', 'Light'] + entrained_spiders]
final_merged_df2 = final_merged_df1.dropna(axis='columns', how='all')  # Ensure no columns with all NaN values are included

cols_list = final_merged_df2.columns.tolist()
final_list = cols_list[3:]

print("The spiders in group B that have data across all times and are entrained are:", final_list)


The spiders in group B that have data across all times and are entrained are: ['Sp1', 'Sp3', 'Sp5', 'Sp6', 'Sp7', 'Sp8', 'Sp11', 'Sp12', 'Sp13', 'Sp14', 'Sp15', 'Sp17', 'Sp18', 'Sp19', 'Sp20', 'Sp21', 'Sp22', 'Sp23']


In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

def process_file(file):
    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]

    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    return df

def process_files(files):
    dataframes = {}
    for idx, file in enumerate(files, start=1):
        df = process_file(file)
        dataframes[f'df{idx}'] = df
    return dataframes

def filter_and_identify_spiders(dataframes, threshold=0):
    spiders_threshold_met = []
    
    for name, df in dataframes.items():
        filtered_spiders = set()
        for day in range(1, 9):  
            day_df = df[df['Day'] == day] 
            count_mov = day_df.filter(like="Sp")  
            sum_mov = count_mov.sum(axis=0)  
            filter_mov = sum_mov > threshold  
            columns_to_keep = filter_mov[filter_mov].index 
            filtered_spiders.update(columns_to_keep)

        spiders_threshold_met.append(filtered_spiders)
    
    common_spiders = set.intersection(*spiders_threshold_met)
    return common_spiders

def entrainment(data, column):
    dflight = data[data['Light'] == 1][column]
    dfdark = data[data['Light'] == 0][column]
    
    lightmean = np.mean(dflight)
    darkmean = np.mean(dfdark)
    if darkmean == 0:
        return False
    
    diff = lightmean / darkmean
    return diff > 0.25

def filter_and_merge(dataframes, common_spiders):
    final_filtered_dfs = []
    
    for name, df in dataframes.items():
        filtered_df = df[['Day', 'Time', 'Light'] + list(common_spiders)]
        filtered_df = filtered_df.dropna(axis=1, how='all')
        final_filtered_dfs.append(filtered_df)
    
    final_merged_df = pd.concat(final_filtered_dfs)
    return final_merged_df

files = ['Steatoda B masking 04 pm.txt', 'Steatoda B masking 10 pm.txt', 'Steatoda B masking 12 pm.txt', 'Steatoda B maskng 2am.txt']

dataframes = process_files(files)
common_spiders = filter_and_identify_spiders(dataframes, threshold=0)
final_merged_df = filter_and_merge(dataframes, common_spiders)

entrained_spiders = []
for spider in common_spiders:
    if entrainment(final_merged_df, spider):
        entrained_spiders.append(spider)

final_merged_df1 = final_merged_df[['Day', 'Time', 'Light'] + entrained_spiders]
final_merged_df2 = final_merged_df1.dropna(axis='columns', how='all')

cols_list = final_merged_df2.columns.tolist()
final_list = cols_list[3:]

print("The spiders in group B that have data across all times and are entrained are:", final_list)

The spiders in group B that have data across all times and are entrained are: ['Sp17', 'Sp5', 'Sp18', 'Sp1', 'Sp7', 'Sp19', 'Sp23', 'Sp3', 'Sp13', 'Sp14', 'Sp11', 'Sp6', 'Sp12', 'Sp21', 'Sp20', 'Sp8', 'Sp22']
